In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns 
import geopandas as gpd
import folium
import warnings
warnings.filterwarnings("ignore")
import re

/home/student.unimelb.edu.au/xinyij4/.local/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
data_all = pd.read_csv("Iowa_Liquor_Sales.csv")
data_all.head(5)

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S29198800001,11/20/2015,2191,Keokuk Spirits,1013 MAIN,KEOKUK,52632,"1013 MAIN\nKEOKUK 52632\n(40.39978, -91.387531)",56.0,Lee,...,297,Templeton Rye w/Flask,6,750,$18.09,$27.14,6,$162.84,4.50,1.19
1,S29195400002,11/21/2015,2205,Ding's Honk And Holler,900 E WASHINGTON,CLARINDA,51632,"900 E WASHINGTON\nCLARINDA 51632\n(40.739238, ...",73.0,Page,...,297,Templeton Rye w/Flask,6,750,$18.09,$27.14,12,$325.68,9.00,2.38
2,S29050300001,11/16/2015,3549,Quicker Liquor Store,1414 48TH ST,FORT MADISON,52627,"1414 48TH ST\nFORT MADISON 52627\n(40.624226, ...",56.0,Lee,...,249,Disaronno Amaretto Cavalli Mignon 3-50ml Pack,20,150,$6.40,$9.60,2,$19.20,0.30,0.08
3,S28867700001,11/04/2015,2513,Hy-Vee Food Store #2 / Iowa City,812 S 1ST AVE,IOWA CITY,52240,812 S 1ST AVE\nIOWA CITY 52240\n,52.0,Johnson,...,237,Knob Creek w/ Crystal Decanter,3,1750,$35.55,$53.34,3,$160.02,5.25,1.39
4,S29050800001,11/17/2015,3942,Twin Town Liquor,104 HIGHWAY 30 WEST,TOLEDO,52342,"104 HIGHWAY 30 WEST\nTOLEDO 52342\n(41.985887,...",86.0,Tama,...,249,Disaronno Amaretto Cavalli Mignon 3-50ml Pack,20,150,$6.40,$9.60,2,$19.20,0.30,0.08


In [3]:
print("The entire dataset dimension: ",data_all.shape) # (12591077, 24)
print("Attributes:",data_all.columns) # (12591077, 24)

The entire dataset dimension:  (12591077, 24)
Attributes: Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)'],
      dtype='object')


In [4]:
# check missing values 
print("Missing values:\n",data_all.isnull().sum()) 

Missing values:
 Invoice/Item Number          0
Date                         0
Store Number                 0
Store Name                   0
Address                   2376
City                      2375
Zip Code                  2420
Store Location            2375
County Number            79178
County                   79178
Category                  8020
Category Name            16086
Vendor Number                3
Vendor Name                  1
Item Number                  0
Item Description             0
Pack                         0
Bottle Volume (ml)           0
State Bottle Cost           10
State Bottle Retail         10
Bottles Sold                 0
Sale (Dollars)              10
Volume Sold (Liters)         0
Volume Sold (Gallons)        0
dtype: int64


In [5]:
# drop records with missing values 
data_all = data_all.dropna()

In [6]:
print("##################\n",data_all.groupby("City").size()) # 790 cities
print("##################\n",data_all.groupby("County").size()) # 200 cities
print("##################\n",data_all.groupby("Item Number").size())# 7263 brands

##################
 City
ACKLEY       2722
ADAIR        1567
ADEL        10878
AFTON         994
AKRON        2094
            ...  
Winthrop      994
Woodbine     2770
Woodward      605
ZWINGLE      1260
Zwingle       455
Length: 790, dtype: int64
##################
 County
ADAIR           2503
ADAMS           2066
ALLAMAKEE      11260
APPANOOSE      10076
AUDUBON         2143
               ...  
Winnebago      33984
Winneshiek     55567
Woodbury      287930
Worth          12643
Wright         27719
Length: 200, dtype: int64
##################
 Item Number
101       31
102       31
104       10
107       31
108       39
          ..
996026     3
997085     1
997086     3
997878     3
999275     1
Length: 7263, dtype: int64


In [7]:
l = data_all.loc[:, ["Volume Sold (Liters)", "Category"]].groupby("Category").sum()
l = l.sort_values(by = "Volume Sold (Liters)", ascending = False).reset_index()
l.iloc[:3, 0].to_list()

[1031080.0, 1012100.0, 1062310.0]

In [8]:
top_three_category = l.iloc[:3, 0].to_list()
data = data_all.loc[data_all["Category"].isin(top_three_category),:]
data.Category.value_counts()
print(data.shape)

(2981686, 24)


### Pre-processing 

In [9]:
#pd.set_option('display.max_rows', 10)
data["County"] = data["County"].str.title()
data["City"] = data["City"].str.title()
county_countN_df = data.loc[:,["County","County Number","Date"]].groupby(["County","County Number"]).count().reset_index()
county_countN_df["County Number"].value_counts()
print("Counties with the same county number but conflicting names")
print(county_countN_df.loc[county_countN_df["County Number"].isin([71.0, 78.0,17.0, 11.0]),:])

data.County = data.County.replace(["buena vist","cerro gord","obrien","pottawatta"], 
                                  ["buena vista","cerro gordo","o'brien","pottawattamie"])


Counties with the same county number but conflicting names
           County  County Number   Date
10     Buena Vist           11.0   2807
11    Buena Vista           11.0  25146
17     Cerro Gord           17.0   6669
18    Cerro Gordo           17.0  65819
72        O'Brien           71.0  21350
73         Obrien           71.0   3086
80     Pottawatta           78.0   8204
81  Pottawattamie           78.0  84674


In [10]:
# convert data types 
data["Date"]  = pd.to_datetime(data['Date'],format='%m/%d/%Y')
data['year'] = pd.DatetimeIndex(data['Date']).year
data['month'] = pd.DatetimeIndex(data['Date']).month
data['day'] = pd.DatetimeIndex(data['Date']).day


data.loc[:,"Sale (Dollars)"] = data.loc[:, "Sale (Dollars)"].str[1:]
data.loc[:,"State Bottle Cost"] = data.loc[:, "State Bottle Cost"].str[1:]
data.loc[:,"State Bottle Retail"] = data.loc[:, "State Bottle Retail"].str[1:]
data["Store Number"] = data["Store Number"].astype("category")
data["County Number"] = data["County Number"].astype("int")
data["County Number"] = data["County Number"].astype("category")
data["County"] = data["County"].astype("category")
data["City"] = data["City"].astype("category")
data["Sale (Dollars)"] = data["Sale (Dollars)"].astype("float")
data["State Bottle Cost"] = data["State Bottle Cost"].astype("float")
data["State Bottle Retail"] = data["State Bottle Retail"].astype("float")

In [11]:
# remove potential errors 
cond1 = data["Bottle Volume (ml)"] == 0
cond2 = data["State Bottle Cost"] == 0
cond3 = data["State Bottle Retail"] == 0
cond4 = data["Sale (Dollars)"] == 0
cond5 = data["Bottles Sold"] ==0
cond6 = data["Volume Sold (Liters)"] ==0
data = data.loc[~(cond1|cond2|cond3|cond4|cond5|cond6),:] # 640 rows removed

In [13]:
data["latitude"]  = [re.findall(r"\((.+?),",i) for i in data["Store Location"]]
data["longitude"]  = [re.findall(r", (.+?)\)",i) for i in data["Store Location"]]
# 2981046 (about 600 records with missing coordinates)
filtered_coord = [len(data.iloc[i, 28]) != 0 for i in range(data.shape[0])]
filtered_coord
data_filtered = data.loc[filtered_coord, :]
data_filtered.latitude = data_filtered.latitude.map(lambda x:x[0])
data_filtered.longitude = data_filtered.longitude.map(lambda x:x[0])
data_filtered.latitude = data_filtered.latitude.astype("float")
data_filtered.longitude = data_filtered.longitude.astype("float")

In [14]:
data_filtered.County = data_filtered.County.str.title()

data_filtered["Profit"] = (data_filtered["State Bottle Retail"] - 
                           data_filtered["State Bottle Cost"])*data_filtered["Bottles Sold"]

### Visualisation 

In [73]:
sf = gpd.read_file("county.shp")
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
gdf_PUL = gpd.GeoDataFrame(pd.merge(data_filtered, sf, left_on = 'County', 
                                right_on = 'COUNTY')).drop('County', axis = 1)

geoJSON = gdf_PUL[['geometry','COUNTY']].drop_duplicates('COUNTY').to_json()
pd.set_option('display.max_rows', 103)
data_filtered.County.value_counts()

# Analyze total alcohol sales in different counties (chorolength)
volume_sold_df = data_filtered.loc[:, ["County", "Volume Sold (Liters)"]].groupby("County").sum().reset_index() # 103 counties

profit_df = data_filtered.loc[:, ["County", "Profit"]].groupby("County").sum().reset_index()


td_map = folium.Map(location = [41.8780, -93.0977], 
                    zoom_start= 10,tiles = "cartodbpositron")

cho_map = folium.Choropleth(
    geo_data=geoJSON,
    data=volume_sold_df[['County','Volume Sold (Liters)']],
    name='choropleth',
    fill_color='YlGn',
    fill_opacity=0.7,
    key_on = 'feature.properties.COUNTY',
    line_opacity=0.2,
    legend_name='Volume Sold (Liters)',
    highlight=True,
    line_color='green',
    columns = ['County','Volume Sold (Liters)'], reset = True,
).add_to(td_map)

cho_map.geojson.add_child(folium.features.GeoJsonTooltip(['COUNTY'], labels = False))
folium.LayerControl().add_to(td_map)
td_map.save('foliumChoropleth_map_volume_sold.html')
td_map

In [ ]:
td_map = folium.Map(location = [41.8780, -93.0977], 
                    zoom_start= 10,tiles = "cartodbpositron")

cho_map = folium.Choropleth(
    geo_data=geoJSON,
    data=profit_df[['County','Profit']],
    name='choropleth',
    fill_color='YlGn',
    fill_opacity=0.7,
    key_on = 'feature.properties.COUNTY',
    line_opacity=0.2,
    legend_name='Profit',
    highlight=True,
    line_color='green',
    columns = ['County','Profit'], reset = True,
).add_to(td_map)

cho_map.geojson.add_child(folium.features.GeoJsonTooltip(['COUNTY'], labels = False))
folium.LayerControl().add_to(td_map)
#td_map.save('foliumChoroplethMap_trip_distance.html')
td_map